In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [2]:
#train_df = pd.read_csv("./train_df_encode.csv")
train_df = pd.read_csv("./data/train.tsv", delimiter='\t')
#train_df.info()

In [3]:
train_df.head(10)

,id,Y,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,3,12.36,3.83,2.38,21.0,88,2.30,0.92,0.50,1.04,7.65,0.56,1.58,520
1,1,2,12.42,4.43,2.73,26.5,102,2.20,2.13,0.43,1.71,2.08,0.92,3.12,365
2,3,1,13.05,1.77,2.10,17.0,107,3.00,3.00,0.28,2.03,5.04,0.88,3.35,885
3,6,1,13.05,1.65,2.55,18.0,98,2.45,2.43,0.29,1.44,4.25,1.12,2.51,1105
4,9,1,13.05,2.05,3.22,25.0,124,2.63,2.68,0.47,1.92,3.58,1.13,3.20,830
5,11,3,12.93,2.81,2.70,21.0,96,1.54,0.50,0.53,0.75,4.60,0.77,2.31,600
6,12,3,13.36,2.56,2.35,20.0,89,1.40,0.50,0.37,0.64,5.60,0.70,2.47,780
7,13,2,12.29,1.41,1.98,16.0,85,2.55,2.50,0.29,1.77,2.90,1.23,2.74,428
8,14,1,13.07,1.50,2.10,15.5,98,2.40,2.64,0.28,1.37,3.70,1.18,2.69,1020
9,15,1,13.74,1.67,2.25,16.4,118,2.60,2.90,0.21,1.62,5.85,0.92,3.20,1060


In [4]:
train_data = train_df.values
xs = train_data[:, 2:]  # 正解データ列以外を抽出
y = train_data[:, 1]  # 正解データ列のみ抽出

In [5]:
y = y.reshape(-1,1)

In [6]:
y = OneHotEncoder().fit_transform(y).toarray()

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [7]:
y

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0

In [8]:
xs.shape

(89, 13)

In [9]:
y.shape

(89, 3)

In [10]:
xs = preprocessing.scale(xs)
xs[0,]

array([-0.77842615,  1.36759091, -0.02518151,  0.34823926, -0.88141297,
       -0.03356369, -1.13000805,  1.10535029, -0.90189213,  1.08677281,
       -1.78586733, -1.48085902, -0.71221127])

In [11]:
#y[y[0,] == 1].sum() / y.size
class_count = y.shape[1]
class_count

3

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, input_dim=xs.shape[1], activation='tanh'))
#model.add(Dropout(0.5))
model.add(Dense(64, activation='tanh'))
#model.add(Dropout(0.5))
model.add(Dense(class_count, activation='softmax'))

model.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
#              loss='binary_crossentropy',
              metrics=['accuracy'])

#model.fit(xs, y, epochs=5)
#loss, accuracy = model.evaluate(xs, y)

model.fit(xs, y, epochs=20)
loss, accuracy = model.evaluate(xs, y)

print("loss = {:.2f}".format(loss))
print("acc = {:.2f}".format(accuracy))

Epoch 1/20
89/89 [==============================] - 1s 8ms/step - loss: 1.1250 - acc: 0.4719
Epoch 2/20
89/89 [==============================] - 0s 218us/step - loss: 0.7651 - acc: 0.7640
Epoch 3/20
89/89 [==============================] - 0s 233us/step - loss: 0.5290 - acc: 0.8989
Epoch 4/20
89/89 [==============================] - 0s 152us/step - loss: 0.3789 - acc: 0.9213
Epoch 5/20
89/89 [==============================] - 0s 178us/step - loss: 0.2900 - acc: 0.9438
Epoch 6/20
89/89 [==============================] - 0s 210us/step - loss: 0.2316 - acc: 0.9438
Epoch 7/20
89/89 [==============================] - 0s 223us/step - loss: 0.1920 - acc: 0.9551
Epoch 8/20
89/89 [==============================] - 0s 231us/step - loss: 0.1631 - acc: 0.9551
Epoch 9/20
89/89 [==============================] - 0s 201us/step - loss: 0.1400 - acc: 0.9551
Epoch 10/20
89/89 [==============================] - 0s 183us/step - loss: 0.1209 - acc: 0.9775
Epoch 11/20
89/89 [==============================] 

In [13]:
test_df = pd.read_csv("./data/test.tsv", delimiter='\t')
#test_df = pd.read_csv("./data/train.tsv", delimiter='\t')
#test_df.info()

In [14]:
test_df.head(5)

,id,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,2,14.10,2.16,2.30,18.0,105,2.95,3.32,0.22,2.38,5.75,1.25,3.17,1510
1,4,13.23,3.30,2.28,18.5,98,1.80,0.83,0.61,1.87,10.52,0.56,1.51,675
2,5,12.08,1.13,2.51,24.0,78,2.00,1.58,0.40,1.40,2.20,1.31,2.72,630
3,7,12.08,1.83,2.32,18.5,81,1.60,1.50,0.52,1.64,2.40,1.08,2.27,480
4,8,14.30,1.92,2.72,20.0,120,2.80,3.14,0.33,1.97,6.20,1.07,2.65,1280


In [15]:
test_data = test_df.values
xs_test = test_data[:, 1:]
#xs_test = test_data[:, 2:]

In [16]:
xs_test[0]

array([1.41e+01, 2.16e+00, 2.30e+00, 1.80e+01, 1.05e+02, 2.95e+00,
       3.32e+00, 2.20e-01, 2.38e+00, 5.75e+00, 1.25e+00, 3.17e+00,
       1.51e+03])

In [17]:
xs_test = preprocessing.scale(xs_test)

In [18]:
xs_test 

array([[ 1.3688573 , -0.2129697 , -0.19058258, ...,  1.33841217,
         0.76173411,  2.44330849],
       [ 0.28210839,  0.831395  , -0.27455213, ..., -1.70528103,
        -1.53216312, -0.2357689 ],
       [-1.15439878, -1.15656237,  0.69109771, ...,  1.60308114,
         0.1398945 , -0.38015032],
       ...,
       [ 0.49446163,  2.0223372 ,  2.16056485, ..., -1.22005458,
        -0.96559814, -0.38015032],
       [-0.79214914, -1.21152893, -1.03027809, ...,  0.41207076,
         0.20898779, -0.28389604],
       [-0.71720094, -0.79011861, -0.23256735, ..., -1.13183159,
         0.3057184 , -1.27210663]])

In [19]:
#y_test = model.predict(xs_test, verbose=1)
y_test = model.predict_proba(xs_test, verbose=1)

89/89 [==============================] - 0s 359us/step


In [20]:
y_test.shape

(89, 3)

In [21]:
y_test[0:20,:]

array([[9.9835938e-01, 6.2015880e-04, 1.0204142e-03],
       [6.3670422e-03, 1.6171603e-03, 9.9201584e-01],
       [3.3467924e-04, 9.9899489e-01, 6.7041873e-04],
       [4.3389029e-04, 9.9789017e-01, 1.6760513e-03],
       [9.8615175e-01, 2.9269996e-04, 1.3555533e-02],
       [9.8815465e-01, 8.1035001e-03, 3.7417801e-03],
       [3.8545225e-03, 9.9595058e-01, 1.9477683e-04],
       [3.3807740e-01, 6.2939984e-01, 3.2522686e-02],
       [9.8219371e-01, 5.8309287e-03, 1.1975356e-02],
       [7.2318222e-04, 9.9824631e-01, 1.0304964e-03],
       [9.9480999e-01, 2.2235165e-04, 4.9676546e-03],
       [6.0672844e-03, 9.5505238e-04, 9.9297768e-01],
       [9.9354279e-01, 2.0267630e-03, 4.4304263e-03],
       [3.4373926e-04, 9.9677306e-01, 2.8831826e-03],
       [1.3718385e-03, 6.3100150e-03, 9.9231815e-01],
       [9.9815947e-01, 2.3447677e-04, 1.6060197e-03],
       [8.2551325e-03, 3.8382277e-04, 9.9136108e-01],
       [9.1434979e-01, 4.9996939e-02, 3.5653193e-02],
       [3.1734575e-03, 2.947

In [22]:
#y_test_class = tf.keras.backend.argmax(y_test,axis=1)
y_test_class = np.argmax(y_test, axis = 1) + 1

In [23]:
y_test_class.shape

(89,)

In [24]:
y_test_class

array([1, 3, 2, 2, 1, 1, 2, 2, 1, 2, 1, 3, 1, 2, 3, 1, 3, 1, 3, 2, 3, 2,
       2, 2, 2, 2, 3, 3, 2, 2, 1, 3, 3, 1, 1, 2, 3, 2, 3, 2, 3, 3, 2, 1,
       3, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 3, 3, 3, 1, 2, 1, 1, 1, 2, 1, 1,
       1, 1, 3, 2, 3, 2, 1, 2, 3, 1, 1, 2, 2, 3, 1, 1, 1, 2, 1, 2, 3, 2,
       2])

In [25]:
Y_concat = pd.DataFrame()
Y_concat["id"] = test_df["id"].astype(int)
Y_concat["Y"] = y_test_class.astype(int)

In [29]:
Y_concat.to_csv("predict_result_data1.csv", header=False, index=False)

In [27]:
#from sklearn.metrics import accuracy_score
#acc = accuracy_score(train_df['Y'],y_test_class)
#acc

In [28]:
#train_df['Y'].values